In [1]:
!pip install qwen_vl_utils
!pip install transformers
!pip install datasets
!pip install -U bitsandbytes
!pip install trl

import torch
from transformers import (
    Qwen2VLForConditionalGeneration,
    AutoProcessor,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig,
    Qwen2VLProcessor,
    AutoTokenizer,
    AutoImageProcessor
)
from trl import SFTConfig
from qwen_vl_utils import process_vision_info
from datasets import load_dataset#, Dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import os
import wandb
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
import gc
import time
from typing import Dict, List


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 30.6 MB/s eta 0:00:00
  

In [2]:
from google.colab import drive
drive.mount('/content/drive')

compute = True

# Check device compatibility
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
print('device = ', device)

model_name = "Qwen/Qwen2-VL-2B-Instruct"

img_folder_path = "/content/drive/MyDrive/MATH-V-main"

dataset_name = "MathLLMs/MathVision"

file_name = f"data/output/{dataset_name.split('/')[-1]}_{model_name.split('/')[-1]}.csv"


# Load the dataset
data_list = load_dataset(dataset_name, split='test')


# Split data into training and validation sets
train_size = int(0.7 * len(data_list))
val_size = int(0.1 * len(data_list))
test_size = len(data_list) - train_size - val_size
data_list = data_list.shuffle(seed=42)  # Shuffle the dataset for randomness

train_data = data_list.select(range(train_size))
val_data = data_list.select(range(train_size, train_size + val_size))
test_data = data_list.select(range(train_size+val_size, len(data_list)))


Mounted at /content/drive
device =  cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

(…)-00000-of-00001-3532b8d3f1b4047a.parquet:   0%|          | 0.00/57.0M [00:00<?, ?B/s]

(…)-00000-of-00001-f8ff70fcb2f29b1d.parquet:   0%|          | 0.00/6.99M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/3040 [00:00<?, ? examples/s]

Generating testmini split:   0%|          | 0/304 [00:00<?, ? examples/s]

In [3]:
def generate_message(inputs):
    messages = []
    for input in inputs:
        prompt = input["question"]
        image_path = f"{img_folder_path}/{input['image']}"
        messages.append({
            "role": "user",
            "content": [{"type": "image", "image": image_path}, {"type": "text", "text": prompt}]
        })

    return messages

def generate_embedding(model, processor, sample):
    messages = generate_message([sample])
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )

    # Move inputs to the device
    inputs = inputs.to(device)
    return inputs

def generate_text_from_sample(model, processor, sample, max_new_tokens=1024):
    inputs = generate_embedding(model, processor, sample)

    # Move model to the device
    model = model.to(device)

    # Perform inference
    generated_ids = model.generate(**inputs, max_new_tokens=256)

    # Trim the generated ids to remove the input ids
    trimmed_generated_ids = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(model_inputs.input_ids, generated_ids)]

    output_text = processor.batch_decode(
        trimmed_generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]
    return output_text


In [4]:
# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
try:
  # raise Exception('just download')
  model = Qwen2VLForConditionalGeneration.from_pretrained(
      f"{img_folder_path}/{model_name}",
      # device_map="auto",
      torch_dtype=torch.bfloat16,
      # quantization_config=bnb_config
  )

except:
  model = Qwen2VLForConditionalGeneration.from_pretrained(
      model_name,
      # device_map="auto",
      torch_dtype=torch.bfloat16,
      # quantization_config=bnb_config
  )
  model.save_pretrained(f"{img_folder_path}/{model_name}")
processor = Qwen2VLProcessor.from_pretrained(model_name)


# Configure LoRA
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

# Apply PEFT model adaptation
peft_model = get_peft_model(model, peft_config)

# Print trainable parameters
peft_model.print_trainable_parameters()

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

trainable params: 1,089,536 || all params: 2,210,075,136 || trainable%: 0.0493


In [5]:
import torch
from torch.utils.data import Dataset
from typing import List, Dict, Any, Optional
import os
from transformers import Trainer, TrainingArguments

class MessageDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.processor = processor
        if processor is None:
            raise ValueError("Processor cannot be None")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        message = generate_message([self.data[idx]])[0]
        if message is None:
            raise ValueError(f"Generated message at index {idx} is None")
        return message

def collate_fn(examples: List[Any], processor) -> Dict[str, torch.Tensor]:
    """
    Collate function for batch processing of multimodal data.

    Args:
        examples: List of examples to process
        processor: Vision-language processor

    Returns:
        Dict containing batched tensors
    """
    if processor is None:
        raise ValueError("Processor cannot be None")

    if not examples:
        raise ValueError("Examples list is empty")

    # Process texts and images in parallel
    texts = []
    image_inputs = []

    for msg in examples:
        if msg is None:
            raise ValueError("Message in batch cannot be None")

        text = processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True)
        texts.append(text)

        # Safely process vision info
        vision_info = process_vision_info([msg])
        if vision_info and vision_info[0] is not None:
            image_inputs.append(vision_info[0])
        else:
            raise ValueError("Vision info processing failed")

    # Batch process all examples at once
    try:
        batch = processor(
            text=texts,
            images=image_inputs,
            return_tensors="pt",
            padding=True
        )
    except Exception as e:
        raise RuntimeError(f"Error during batch processing: {str(e)}")

    # Create labels and apply masking
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100

    # Handle image token masking
    image_tokens = ([151652, 151653, 151655] if isinstance(processor, Qwen2VLProcessor)
                   else [processor.tokenizer.convert_tokens_to_ids(processor.image_token)])

    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100

    return {
        'input_ids': batch['input_ids'],
        'attention_mask': batch['attention_mask'],
        'pixel_values': batch['pixel_values'],
        'labels': labels
    }

output_dir = f"./fine_tuned_{model_name}"
os.makedirs(output_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=os.path.join(output_dir, 'logs'),
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    learning_rate=2e-5,
    fp16=True,
    gradient_accumulation_steps=8,
    save_total_limit=3,
    remove_unused_columns=False,
    dataloader_num_workers=2,
    gradient_checkpointing=True,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
)

from transformers.models.auto.modeling_auto import (
    MODEL_FOR_CAUSAL_LM_MAPPING_NAMES
)
from transformers.trainer import _is_peft_model

class CustomTrainer(SFTTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if (self.label_smoother is not None or self.compute_loss_func is not None) and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        loss_kwargs = {}
        if self.model_accepts_loss_kwargs:
            if num_items_in_batch is not None:
                loss_kwargs["num_items_in_batch"] = num_items_in_batch
        if inputs['pixel_values'].ndim < 3:
            inputs['pixel_values'] = inputs['pixel_values'].unsqueeze(0)
        loss_kwargs['image_grid_thw'] = torch.tensor([inputs['pixel_values'].shape]).to(inputs['pixel_values'].device)
        inputs = {**inputs, **loss_kwargs}
        print('loss_kwargs = ', loss_kwargs)
        outputs = model(**inputs)
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            unwrapped_model = self.accelerator.unwrap_model(model)
            if _is_peft_model(unwrapped_model):
                model_name = unwrapped_model.base_model.model._get_name()
            else:
                model_name = unwrapped_model._get_name()
            # User-defined compute_loss function
            if self.compute_loss_func is not None:
                loss = self.compute_loss_func(outputs, labels, num_items_in_batch=num_items_in_batch)
            elif model_name in MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.values():
                loss = self.label_smoother(outputs, labels, shift_labels=True)
            else:
                loss = self.label_smoother(outputs, labels)
        else:
            if isinstance(outputs, dict) and "loss" not in outputs:
                raise ValueError(
                    "The model did not return a loss from the inputs, only the following keys: "
                    f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
                )
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        if self.args.average_tokens_across_devices and self.model_accepts_loss_kwargs:
            loss *= self.accelerator.num_processes

        return (loss, outputs) if return_outputs else loss


trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=MessageDataset(train_data),
    eval_dataset=MessageDataset(val_data),
    data_collator=lambda x: collate_fn(x, processor),
    tokenizer=processor.tokenizer
)

# trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-350099bbbd46>:172: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_sid

In [6]:
# Configure training arguments
training_args = SFTConfig(
    output_dir=output_dir,  # Directory to save the model
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size for training
    per_device_eval_batch_size=4,  # Batch size for evaluation
    gradient_accumulation_steps=8,  # Steps to accumulate gradients
    gradient_checkpointing=True,  # Enable gradient checkpointing for memory efficiency
    # Optimizer and scheduler settings
    optim="adamw_torch_fused",  # Optimizer type
    learning_rate=2e-4,  # Learning rate for training
    lr_scheduler_type="constant",  # Type of learning rate scheduler
    # Logging and evaluation
    logging_steps=10,  # Steps interval for logging
    eval_steps=10,  # Steps interval for evaluation
    eval_strategy="steps",  # Strategy for evaluation
    save_strategy="steps",  # Strategy for saving the model
    save_steps=20,  # Steps interval for saving
    metric_for_best_model="eval_loss",  # Metric to evaluate the best model
    greater_is_better=False,  # Whether higher metric values are better
    load_best_model_at_end=True,  # Load the best model after training
    # Mixed precision and gradient settings
    bf16=True,  # Use bfloat16 precision
    tf32=True,  # Use TensorFloat-32 precision
    max_grad_norm=0.3,  # Maximum norm for gradient clipping
    warmup_ratio=0.03,  # Ratio of total steps for warmup
    # Hub and reporting
    push_to_hub=False,  # Whether to push model to Hugging Face Hub
    report_to="wandb",  # Reporting tool for tracking metrics
    # Gradient checkpointing settings
    gradient_checkpointing_kwargs={"use_reentrant": False},  # Options for gradient checkpointing
    # Dataset configuration
    dataset_text_field="",  # Text field in dataset
    dataset_kwargs={"skip_prepare_dataset": True},  # Additional dataset options
    # max_seq_length=1024  # Maximum sequence length for input
)

training_args.remove_unused_columns = False  # Keep unused columns in dataset


wandb.init(
    project="qwen2-7b-instruct-trl-sft-ChartQA",  # change this
    name="qwen2-7b-instruct-trl-sft-ChartQA",  # change this
    config=training_args,
)

# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# # Create datasets
# math_train_data = MathVQADataset(train_data, tokenizer, image_processor)
# math_val_data = MathVQADataset(val_data, tokenizer, image_processor)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=MessageDataset(train_data),
    eval_dataset=MessageDataset(val_data),
    data_collator=lambda x: collate_fn(x, processor),
    peft_config=peft_config,
    tokenizer=processor.tokenizer,
)

trainer.train()

trainer.save_model(training_args.output_dir)



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


<ipython-input-6-91a41ae8b2c6>:52: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


loss_kwargs =  {'image_grid_thw': tensor([[    1, 28988,  1176]], device='cuda:0')}


RuntimeError: The size of tensor a (28988) must match the size of tensor b (34089888) at non-singleton dimension 1

In [ ]:
del trainer  # Free memory after training
# del model    # Free memory after training

def clear_memory():
    # Delete variables if they exist in the current global scope
    if "inputs" in globals():
        del globals()["inputs"]
    if "model" in globals():
        del globals()["model"]
    if "processor" in globals():
        del globals()["processor"]
    if "trainer" in globals():
        del globals()["trainer"]
    if "peft_model" in globals():
        del globals()["peft_model"]
    if "bnb_config" in globals():
        del globals()["bnb_config"]
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")


clear_memory()

In [ ]:
adapter_path = "sergiopaniego/qwen2-7b-instruct-trl-sft-ChartQA"
model.load_adapter(adapter_path)

In [ ]:
# test

# # Load the model
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     model_name,
#     torch_dtype=torch.float32, #if device == "cpu" else torch.bfloat16,
#     device_map=None
# )

# # Initialize processor
# min_pixels = 256 * 28 * 28
# max_pixels = 1280 * 28 * 28
# processor = AutoProcessor.from_pretrained(model_name, min_pixels=min_pixels, max_pixels=max_pixels)


results = []

for i, input in enumerate(tqdm(test_data)):
    output_text = generate_text_from_sample(model, processor, input)

    # Store results
    results.append(input | {
        "generated_text": output_text
    })
    # if i > 4:
    #   break


# pd.DataFrame(results).to_csv(f"{img_folder_path}/test_generated_answer.csv")

In [ ]:

class MathVQADataset(Dataset):
    def __init__(self, data: List[Dict], tokenizer, image_processor, max_length=512, image_size=(448, 448)):  # Updated image size
        self.data = data
        self.tokenizer = tokenizer
        self.image_processor = image_processor
        self.max_length = max_length
        self.image_size = image_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # Load and process image
        image = Image.open(f"{img_folder_path}/{item['image']}").convert('RGB')

        # Process image with all required parameters
        image_features = self.image_processor(
            image,
            return_tensors="pt",
            do_resize=True,
            size={"height": self.image_size[0], "width": self.image_size[1]},
            do_normalize=True
        )

        # Create prompt with system message
        prompt = f"<|im_start|>system\nYou are a helpful assistant that answers math questions based on images.<|im_end|>\n<|im_start|>user\nQuestion: {item['question']}<|im_end|>\n<|im_start|>assistant\nLet me solve this step by step.\nAnswer:"
        target = f"{item['answer']}<|im_end|>"

        # Tokenize input and target
        inputs = self.tokenizer(
            prompt,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        targets = self.tokenizer(
            target,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': targets['input_ids'].squeeze(),
            'pixel_values': image_features['pixel_values'].squeeze(),
        }

def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    pixel_values = torch.stack([item['pixel_values'] for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels,
        'pixel_values': pixel_values
    }

def train_model(train_data: List[Dict], validation_data: List[Dict], model_save_name: str):
    # Initialize model, tokenizer, and image processor with trust_remote_code=True
    model_name = "Qwen/Qwen2-VL-2B-Instruct"
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype="auto", device_map="auto"  # Enable automatic device mapping
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    image_processor = AutoImageProcessor.from_pretrained(model_name, trust_remote_code=True)


    # # Preparation for inference
    # text = processor.apply_chat_template(
    #     messages, tokenize=False, add_generation_prompt=True
    # )
    # image_inputs, video_inputs = process_vision_info(messages)
    # inputs = processor(
    #     text=[text],
    #     images=image_inputs,
    #     videos=video_inputs,
    #     padding=True,
    #     return_tensors="pt",
    # )
    # inputs = inputs.to("cuda")

    # # Inference: Generation of the output
    # generated_ids = model.generate(**inputs, max_new_tokens=128)
    # generated_ids_trimmed = [
    #     out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    # ]
    # output_text = processor.batch_decode(
    #     generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    # )

    # Set output directory
    output_dir = f"./fine_tuned_{model_save_name}"
    os.makedirs(output_dir, exist_ok=True)

    # Create datasets
    train_dataset = MathVQADataset(train_data, tokenizer, image_processor)
    val_dataset = MathVQADataset(validation_data, tokenizer, image_processor)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=2,  # Reduced batch size
        per_device_eval_batch_size=2,   # Reduced batch size
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=os.path.join(output_dir, 'logs'),
        logging_steps=100,
        evaluation_strategy="steps",
        eval_steps=500,
        save_steps=1000,
        learning_rate=2e-5,
        fp16=True,
        gradient_accumulation_steps=8,  # Increased gradient accumulation
        save_total_limit=3,
        remove_unused_columns=False,
        dataloader_num_workers=2
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=collate_fn,
    )

    # Start training
    trainer.train()

    # Save the final model
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)
    image_processor.save_pretrained(output_dir)

# Usage
model_save_name = "math_vqa_model"
train_model(train_data, val_data, model_save_name)

In [ ]:
def generate_prompt(example, prompt_type ='full', context=''):
    question = example['question']
    options = ''
    if len(example['options']) > 0:
        assert len(example['options']) == 5, example
        if ''.join(example['options']) != 'ABCDE':
            options = f"(A) {example['options'][0]}\n(B) {example['options'][1]}\n(C) {example['options'][2]}\n(D) {example['options'][3]}\n(E) {example['options'][4]}\n"

    # input = 'Please solve the problem step by step and put your answer in one "\\boxed{}". If it is a multiple choice question, only one letter is allowed in the "\\boxed{}".\n'+f"{question}\n{options}"
    if prompt_type == 'simple':
      prompt = f"{question}\n{options}\nAnswer the question using a single word or phrase."
    else:
      prompt = f"""{context}
        Question: {question}\n{options}
        """ + 'Please solve the problem by elaborately, providing step-by-step reasoning for the solution and put your answer in one "\\boxed{}. Also, provide the bounding box coordinate of the region that can help you answer the question better.If it is a multiple choice question, only one letter is allowed in the \\boxed{}.\n'
    return prompt

In [ ]:
import re
remove_list = ['the answer', 'is', ':', 'boxed', '$']

results = pd.DataFrame(results)
def final_answer(text: str):
    text = text.lower()
    if 'answer is' not in text:
        return np.nan
    # Create a regex pattern to match all words in the remove_list
    pattern = r'(' + r'|'.join(re.escape(word) for word in remove_list) + r'|\s)'
    # Use re.sub to replace matched patterns with an empty string
    result = re.sub(pattern, '', text.split('answer is')[-1])
    result = re.sub(r'\\\{(\\frac\{[^{}]+\}\{[^{}]+\})\}', r'\1', result)
    return re.sub(r'\\\\', r'\\', result)
# Example metric: String matching (very basic)
results['prediction'] = results['generated_text'].apply(final_answer)
results['exact_match'] = results['prediction'] == results['answer']
display(results)
pd.DataFrame(results).to_csv(f"{img_folder_path}/test_generated_answer.csv")

accuracy = sum(results['exact_match']) / len(results)
print(f"Exact Match Accuracy: {accuracy:.4f}")

# Optionally save results
# import json
# with open("evaluation_results.json", "w") as f:
#     json.dump(results, f, indent=4)